# Projekt końcowy z Metod Numerycznych 2023/2024

Pytania:
- Czy można do raportu użyć markdowna?

Użyto danych z biblioteki meteostat. Wybrano 5 lokalizacji:
- Kraków
- Warszawa
- Wrocław
- Szczecin
- Gdańsk

In [28]:
import pandas as pd
import numpy as np
import matplotlib as mp
from meteostat import Point, Daily, units
from datetime import datetime

In [29]:
cities = {
    'Kraków': Point(50.0647, 19.9450),
    'Warszawa': Point(52.2297, 21.0122),
    'Wrocław': Point(51.1079, 17.0385),
    'Szczecin': Point(53.4285, 14.5528),
    'Gdańsk': Point(54.3520, 18.6466)
}

In [30]:
start = datetime(2024, 1, 1)
end = datetime(2024, 1, 31)

In [31]:
dataframes = []
for city, point in cities.items():
    data = Daily(point, start, end)
    data = data.fetch()
    data['City'] = city
    filename = f'{city.lower()}_weather_data.csv'
    data.to_csv(filename, index=True)
    print(f"Plik CSV dla {city} został pomyślnie wygenerowany jako {filename}.")

Plik CSV dla Kraków został pomyślnie wygenerowany jako kraków_weather_data.csv.
Plik CSV dla Warszawa został pomyślnie wygenerowany jako warszawa_weather_data.csv.
Plik CSV dla Wrocław został pomyślnie wygenerowany jako wrocław_weather_data.csv.
Plik CSV dla Szczecin został pomyślnie wygenerowany jako szczecin_weather_data.csv.
Plik CSV dla Gdańsk został pomyślnie wygenerowany jako gdańsk_weather_data.csv.


### Dane zostały wygenerowane, zatem można przystąpić do obróbki danych

Sprawdzenie, czy występują braki

In [32]:
df = pd.read_csv('kraków_weather_data.csv')

In [33]:
df.isna().sum()

time     0
tavg     0
tmin     0
tmax     0
prcp     0
snow    21
wdir     0
wspd     0
wpgt     0
pres     0
tsun    31
City     0
dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    31 non-null     object 
 1   tavg    31 non-null     float64
 2   tmin    31 non-null     float64
 3   tmax    31 non-null     float64
 4   prcp    31 non-null     float64
 5   snow    10 non-null     float64
 6   wdir    31 non-null     float64
 7   wspd    31 non-null     float64
 8   wpgt    31 non-null     float64
 9   pres    31 non-null     float64
 10  tsun    0 non-null      float64
 11  City    31 non-null     object 
dtypes: float64(10), object(2)
memory usage: 3.0+ KB


Ani jedna dana "The mean sunshine total in minutes [m]" nie została podana, dlatego należy tą pustą kolumnę usunąć.

In [35]:
df.drop(columns=['tsun'], inplace=True)

In [36]:
df.head(5)

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,City
0,2024-01-01,2.7,0.5,4.7,8.9,NaN,260.0,12.8,35.2,1009.2,Kraków
1,2024-01-02,4.9,2.9,7.0,9.9,NaN,193.0,9.7,29.6,1008.0,Kraków
2,2024-01-03,7.4,4.1,10.2,5.5,NaN,223.0,19.2,44.5,997.5,Kraków
3,2024-01-04,6.6,3.1,9.2,7.1,NaN,235.0,25.5,53.7,998.6,Kraków
4,2024-01-05,1.3,0.0,6.5,5.8,NaN,25.0,14.4,46.3,1009.8,Kraków


Wartości puste pojawiają się również w kolumnie "snow", jednakże w tym przypadku kolumna nie jest całkowicie pusta, dlatego też można uzupełnić braki poprzez aproksymację.

In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv('kraków_weather_data.csv')

def moving_average_approximation(series, window=3):
    n = len(series)
    approximated_series = series.copy()
    
    for i in range(n):
        if np.isnan(approximated_series[i]):
            valid_values = []
            
            for j in range(1, window + 1):
                if i - j >= 0 and not np.isnan(approximated_series[i - j]):
                    valid_values.append(approximated_series[i - j])
            
            for j in range(1, window + 1):
                if i + j < n and not np.isnan(approximated_series[i + j]):
                    valid_values.append(approximated_series[i + j])
            
            if len(valid_values) > 0:
                approximated_series[i] = np.mean(valid_values)
    
    return approximated_series

df['snow'] = moving_average_approximation(df['snow'])

df.to_csv('kraków_weather_approximated.csv', index=False)
print("Dane zostały zapisane do pliku 'kraków_weather_approximated.csv'.")


Dane zostały zapisane do pliku 'kraków_weather_approximated.csv'.


Inną metodą uzupełnienia wartości pustych jest interpolacja

In [40]:
df = pd.read_csv('kraków_weather_data.csv')

def jakjakochamtrapezy(func, a, b, n):
    h = (b - a) / n
    suma = 0.0
    for i in range(n):
        suma += func(a + i * h + h / 2) * h
    return suma

def LU_decomposition(A):
    n = len(A)

    L = np.eye(n)
    U = np.zeros((n, n))

    for k in range(n):
        for j in range(k, n):
            sum = 0
            for p in range(k):
                sum += L[k][p] * U[p][j]
            U[k][j] = A[k][j] - sum

        for i in range(k, n):
            sum = 0
            for p in range(k):
                sum += L[i][p] * U[p][k]
            L[i][k] = (A[i][k] - sum) / U[k][k]

    return L, U

def solve_LU(L, U, B):
    n = len(L)
    z = np.zeros(n)
    x = np.zeros(n)

    for i in range(n):
        sum = 0
        for j in range(i):
            sum += L[i][j] * z[j]
        z[i] = B[i] - sum

    for i in range(n - 1, -1, -1):
        sum = 0
        for j in range(i + 1, n):
            sum += U[i][j] * x[j]
        x[i] = (z[i] - sum) / U[i][i]

    return x

def oblicz(a, b, iloscElementow):
    macierz = np.zeros((iloscElementow, iloscElementow))
    ext = np.zeros(iloscElementow)

    for i in range(iloscElementow):
        for j in range(iloscElementow):
            macierz[i][j] = jakjakochamtrapezy(lambda x: pow(x, i + j), a, b, 1000)
        ext[i] = jakjakochamtrapezy(lambda x: Funkcja(x) * pow(x, i), a, b, 1000)

    print("Pierwotna macierz A:")
    print(macierz)

    L, U = LU_decomposition(macierz)

    print("Macierz A po dekompozycji LU:")
    print(macierz)

    print("Wektor B po dekompozycji LU:")
    print(ext)

    return macierz

df['snow'] = oblicz(a, b, iloscElementow)

df.to_csv('kraków_weather_interpolated.csv', index=False)
print("Dane po interpolacji zostały zapisane do pliku 'kraków_weather_interpolated.csv'.")

def WyswietlWyniki(a):
    print("Wyniki wspolczynnikow a:")
    print(a)

df['snow'] = oblicz(df['snow'])

df.to_csv('kraków_weather_interpolated.csv', index=False)
print("Dane po interpolacji zostały zapisane do pliku 'kraków_weather_interpolated.csv'.")



NameError: name 'a' is not defined